In [11]:
import numpy as np
import json
import bokeh
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON, CARTODBPOSITRON_RETINA
from datetime import datetime

import math

from pylab import cm
import matplotlib

In [12]:
def coords_to_plot(lat, lon):
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    x = int(x)
    y = int(y)
    return (x, y)

# Code violation cases

In [29]:
data = pd.read_csv('./seattle-code-violation-cases/code-violation-cases.csv')
data.head()

print('len:', len(data))
print('start:', data['Date Case Created'].min())
print('end:', data['Date Case Created'].max(), '\n')
# print('nulls:')
# print(data.isnull().sum())

data = data.loc[data['Case Type'] == 'TENANT RELOCATION ORDINANCE']
# data.loc[data['Case Type'] == 'TENANT RELOCATION ORDINANCE', 'Date Case Created'].sort_values()

data = data.dropna(subset=['Latitude', 'Longitude'])

# print('nulls:')
# print(data.isnull().sum())

data['year'] = sorted(data['Date Case Created'].apply(lambda x: int(x[:4])))

# for year in data['year'].unique():
#     print(year, len(data.loc[data['year'] == year]))

len: 41485
start: 2003-09-17T00:00:00
end: 2018-04-27T00:00:00 



In [30]:
### Clustering 

import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

# coords = data[['Latitude', 'Longitude']].values

In [121]:
def get_cluster_dict(data, lat_col, lon_col, kms_radius, min_samples):
    
    def get_centermost_point(cluster):
        try:
            centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
            centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
            return tuple(centermost_point)
        except Exception as e:
            #print(e, MultiPoint(cluster))
            return None
        
    def cluster_dict(row):
        return {'lat': row[lat_col], 'lon': row[lon_col], 'weight': row['weight']}
    
    coords = data[[lat_col, lon_col]].values
    data = data[[lat_col, lon_col]].dropna()
    kms_per_radian = 6371.0088
    epsilon = kms_radius / kms_per_radian
    db = DBSCAN(eps=epsilon, min_samples=min_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
    print('Number of clusters: {}'.format(num_clusters))
    
    data['label'] = db.labels_
    #data.groupby('label').count().head(20)
    
    centermost_points = clusters.map(get_centermost_point)
    
    centermost_points.dropna(inplace=True)
    
    lats, lons = zip(*centermost_points)
    rep_points = pd.DataFrame({'lon':lons, 'lat':lats})

    ### This doesn't handle cases where there is no noise. ### (.values[1:])
    
    rs = rep_points.apply(lambda row: data[(data[lat_col]==row['lat']) & (data[lon_col]==row['lon'])].iloc[0], axis=1)
    
    if min_samples > 1:
        rs['count'] = [x[0] for x in data.groupby('label').count().values[1:]]
    else:
        rs['count'] = [x[0] for x in data.groupby('label').count().values]
    rs['weight'] = rs['count'].apply(lambda x: round(x/rs['count'].sum(), 6))

    return list(rs.apply(lambda x: cluster_dict(x), axis=1))

In [72]:
cluster_lat_lon = get_cluster_dict(data, 'Latitude', 'Longitude', .25, 2)
cluster_lat_lon

Number of clusters: 252


[{'lat': 47.66867, 'lon': -122.379616, 'weight': 0.031381},
 {'lat': 47.723974713000004, 'lon': -122.32622810299999, 'weight': 0.00251},
 {'lat': 47.6115, 'lon': -122.32983999999999, 'weight': 0.464017},
 {'lat': 47.725035552, 'lon': -122.344996836, 'weight': 0.013808},
 {'lat': 47.651240217, 'lon': -122.33631552700001, 'weight': 0.001255},
 {'lat': 47.525590911, 'lon': -122.360474321, 'weight': 0.016318},
 {'lat': 47.717174471999996, 'lon': -122.344896099, 'weight': 0.00251},
 {'lat': 47.664912907, 'lon': -122.314724414, 'weight': 0.058368},
 {'lat': 47.711246, 'lon': -122.36680600000001, 'weight': 0.000837},
 {'lat': 47.538326022, 'lon': -122.29514062, 'weight': 0.005021},
 {'lat': 47.534966, 'lon': -122.27533000000001, 'weight': 0.037029},
 {'lat': 47.658041479, 'lon': -122.398227241, 'weight': 0.000837},
 {'lat': 47.569769056000005, 'lon': -122.290700429, 'weight': 0.002929},
 {'lat': 47.688223386000004, 'lon': -122.32162399399999, 'weight': 0.001883},
 {'lat': 47.579487, 'lon': -1

In [43]:
def plot_weights(cluster_lat_lons):

    rs = pd.DataFrame(cluster_lat_lon)

    p = figure(x_range=(-13630000, -13610000), y_range=(6038000, 6046000),
                   x_axis_type="mercator", y_axis_type="mercator")
    p.add_tile(CARTODBPOSITRON_RETINA)

    rs['merc'] = rs.apply(lambda x: coords_to_plot(x['lat'], x['lon']), axis=1).values
    # print(rs.head())

    for i in rs.index:

        x, y, w = rs.loc[i, 'merc'][0], rs.loc[i, 'merc'][1], rs.loc[i, 'weight']

        p.circle(x = x, 
                 y = y,
                 size=w*1000,
                 fill_alpha=.5,
                 line_alpha=0,
                )

    output_notebook()
    show(p)

In [44]:
# plot_weights(cluster_lat_lon)

In [45]:
def plot_by_year(df, year_col, lat_col, lon_col):
    
    p = figure(x_range=(-13630000, -13610000), y_range=(6038000, 6046000),
               x_axis_type="mercator", y_axis_type="mercator")
    p.add_tile(CARTODBPOSITRON_RETINA)
    
    cmap = cm.get_cmap('seismic', len(df[year_col].unique()))
    
    for cmi, year in enumerate(df[year_col].unique()[-5:]): # last 5 years
        
        rgb = cmap(cmi)[:3]
        rgb = matplotlib.colors.rgb2hex(rgb)
        
        print(year)
        
        coords = df.loc[df['year'] == year, [lat_col, lon_col]]
        coords = coords.apply(lambda x: coords_to_plot(x[lat_col], x[lon_col]), axis=1).values
        for coord in coords[:]:
            x, y = coord[0], coord[1]

            p.circle(x = x, 
                     y = y,
                     size=30,
                     fill_alpha=.1,
                     line_alpha=0,
                     fill_color = rgb
                    )
            
    output_notebook()
    show(p)

In [46]:
# plot_by_year(data, 'year', 'Latitude', 'Longitude')

# SPD Reports

In [75]:
data = pd.read_csv('./SPD_Reports.csv')
data['Offence Start Date'] = pd.to_datetime(data['Offense Start Date'])

print('len:', len(data))
print('start:', data['Offence Start Date'].min())
print('end:', data['Offence Start Date'].max(), '\n')
print('nulls:')
print(data.isnull().sum())

len: 684472
start: 1965-01-10 00:00:00
end: 2016-09-19 13:00:00 

nulls:
Offense Type                0
Offense Description         1
Report Date                 1
Offense Start Date          1
Offense End Date       332148
Block                       1
District                  503
Beat                      497
2000 Census Tract        1813
Longitude                   1
Latitude                    1
Offence Start Date          1
dtype: int64


In [76]:
# data['Offense Type'].value_counts()

In [77]:
# data.head()

In [78]:
# data.loc[data['Offense Type'] == 'THEFT-OTH']

In [79]:
cluster_lat_lon = get_cluster_dict(data.loc[data['Offense Type'] == 'THEFT-OTH'], 'Latitude', 'Longitude', .1, 30)
plot_weights(cluster_lat_lon)

Number of clusters: 128


Loading BokehJS ...

# 911 Incidents

In [80]:
# https://www.kaggle.com/sohier/seattle-police-department-911-incident-response (380 MB)

data = pd.read_csv('/Users/sam/Downloads/Seattle_Police_Department_911_Incident_Response.csv')

data.drop(columns=['Initial Type Description', 
                   'Initial Type Subgroup', 
                   'Initial Type Group', 
                   'At Scene Time'], inplace=True)

print('len:', len(data))

# Downsample
rand_ind = np.random.choice(data.index, 5000)
data = data.loc[rand_ind]

print('Downsample len:', len(data))
data['Event Clearance Date'] = pd.to_datetime(data['Event Clearance Date'])

print('start:', data['Event Clearance Date'].min())
print('end:', data['Event Clearance Date'].max(), '\n')
print('nulls:')
print(data.isnull().sum())

/Users/sam/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


len: 1433853
Downsample len: 5000
start: 2010-05-06 09:04:00
end: 2017-08-29 11:44:01 

nulls:
CAD CDW ID                      0
CAD Event Number                0
General Offense Number          0
Event Clearance Code           32
Event Clearance Description    32
Event Clearance SubGroup       32
Event Clearance Group          32
Event Clearance Date           33
Hundred Block Location          9
District/Sector                 4
Zone/Beat                       0
Census Tract                    9
Longitude                       0
Latitude                        0
Incident Location               0
dtype: int64


In [81]:
data['year'] = data['Event Clearance Date'].apply(lambda x: x.year)

In [82]:
data

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,Zone/Beat,Census Tract,Longitude,Latitude,Incident Location,year
924292,1581973,15000361249,2015361249,176.0,LIQUOR VIOLATION - ADULT,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,2015-10-15 19:12:00,BROADWAY E / E JOHN ST,E,E1,7500.5004,-122.320875,47.619905,"(47.619905164, -122.320874901)",2015.0
1391095,1116814,15000194334,2015194334,176.0,LIQUOR VIOLATION - ADULT,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,2015-06-10 18:20:00,27XX BLOCK OF ALKI AV SW,W,W1,9701.2000,-122.409990,47.579317,"(47.579317222, -122.409989591)",2015.0
489362,857931,12000237997,2012237997,53.0,BURGLARY - UNOCCUPIED STRUCTURE ON RESIDENTIAL...,RESIDENTIAL BURGLARIES,BURGLARY,2012-07-23 12:18:00,9XX BLOCK OF N 34TH ST,J,J3,5400.3008,-122.346092,47.649041,"(47.649040967, -122.346092304)",2012.0
608207,1752369,16000195309,2016195309,177.0,LIQUOR VIOLATION - INTOXICATED PERSON,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,2016-06-03 00:37:45,1 AV / STEWART ST,M,M1,8100.3006,-122.341350,47.610275,"(47.610275, -122.34135)",2016.0
1316172,1025293,15000063226,201563226,350.0,HAZARDS,HAZARDS,HAZARDS,2015-02-25 00:41:00,NE PACIFIC PL / MONTLAKE BV NE,U,U3,5302.3054,-122.303908,47.651083,"(47.65108272, -122.303907725)",2015.0
1303844,659235,12000258180,2012258180,465.0,PEDESTRIAN VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,2012-08-06 11:27:00,15XX BLOCK OF 3RD AVE,M,M3,8100.2008,-122.338247,47.610246,"(47.610246301, -122.338247307)",2012.0
1251673,2211579,14000396163,2014396163,280.0,SUSPICIOUS PERSON,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,2014-11-27 14:21:00,6 AV S / S HOLGATE ST,NaN,OS,NaN,-122.325975,47.586175,"(47.58617472, -122.325974779)",2014.0
6161,22028,10000255647,2010255647,280.0,SUSPICIOUS PERSON,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,2010-07-24 20:29:00,53XX BLOCK OF 34TH AVE NW,B,B2,3200.3015,-122.400748,47.667484,"(47.66748449, -122.400747861)",2010.0
366785,725079,12000034761,201234761,430.0,ACCIDENT INVESTIGATION,TRAFFIC RELATED CALLS,ACCIDENT INVESTIGATION,2012-02-04 19:13:00,NE NORTHGATE WY / ROOSEVELT WY NE,L,L2,1200.3000,-122.317853,47.708560,"(47.708560425, -122.31785262)",2012.0
209617,460961,11000330777,2011330777,53.0,BURGLARY - UNOCCUPIED STRUCTURE ON RESIDENTIAL...,RESIDENTIAL BURGLARIES,BURGLARY,2011-10-08 17:31:00,5XX BLOCK OF BROADWAY E,C,C1,7400.1005,-122.320933,47.623699,"(47.623698979, -122.320933325)",2011.0


In [84]:
# data['Event Clearance Description'].value_counts()

In [86]:
data.loc[data['Event Clearance Description'] == 'SUSPICIOUS PERSON']

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,Zone/Beat,Census Tract,Longitude,Latitude,Incident Location,year
1251673,2211579,14000396163,2014396163,280.0,SUSPICIOUS PERSON,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,2014-11-27 14:21:00,6 AV S / S HOLGATE ST,NaN,OS,NaN,-122.325975,47.586175,"(47.58617472, -122.325974779)",2014.0
6161,22028,10000255647,2010255647,280.0,SUSPICIOUS PERSON,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,2010-07-24 20:29:00,53XX BLOCK OF 34TH AVE NW,B,B2,3200.3015,-122.400748,47.667484,"(47.66748449, -122.400747861)",2010.0
333817,689333,11000399780,2011399780,280.0,SUSPICIOUS PERSON,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,2011-12-13 20:18:00,N 64 ST / WOODLAND PL N,J,J1,3500.2000,-122.347681,47.675316,"(47.675316047, -122.347681021)",2011.0
1057429,2007763,17000187600,2017187600,280.0,SUSPICIOUS PERSON,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,2017-05-27 01:48:30,30XX BLOCK OF NE 127 ST,L,L3,100.5004,-122.295250,47.721092,"(47.721092, -122.29525)",2017.0
1226740,2184903,14000355161,2014355161,280.0,SUSPICIOUS PERSON,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,2014-10-23 18:34:00,20XX BLOCK OF N 80 ST,J,J1,2700.4007,-122.333736,47.686912,"(47.686911869, -122.333735861)",2014.0
471179,837376,12000205851,2012205851,280.0,SUSPICIOUS PERSON,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,2012-06-30 00:11:00,55XX BLOCK OF SW CHARLESTOWN ST,W,W2,9701.4002,-122.403327,47.570326,"(47.570326317, -122.403326675)",2012.0
1195657,2149276,14000295301,2014295301,280.0,SUSPICIOUS PERSON,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,2014-09-04 11:59:00,4 AV NW / NW 120 ST,B,B1,500.2017,-122.362261,47.716065,"(47.716065195, -122.362261346)",2014.0
1278182,2244680,15000010894,201510894,280.0,SUSPICIOUS PERSON,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,2015-01-10 23:15:00,EDGAR MARTINEZ DR S / OCCIDENTAL AV S,K,K3,9300.2031,-122.333263,47.590310,"(47.590309827, -122.333263045)",2015.0
403178,764052,12000094273,201294273,280.0,SUSPICIOUS PERSON,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,2012-03-28 19:59:00,27XX BLOCK OF QUEEN ANNE AVE N,Q,Q2,6000.3005,-122.356885,47.644541,"(47.644540609, -122.356885242)",2012.0
619368,1845744,16000355742,2016355742,280.0,SUSPICIOUS PERSON,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,2016-10-01 04:37:55,52XX BLOCK OF BALLARD AV NW,B,B1,4700.4016,-122.382805,47.665707,"(47.665707, -122.382805)",2016.0


In [87]:
# plot_by_year(data, 'year', 'Latitude', 'Longitude')

In [88]:
cluster_lat_lon = get_cluster_dict(data.loc[data['Event Clearance Description'] == 'SUSPICIOUS PERSON'], 'Latitude', 'Longitude', .1, 2)
plot_weights(cluster_lat_lon)

Number of clusters: 63


Loading BokehJS ...

# King County House Sales

In [89]:
data = pd.read_csv('./kc_house_data.csv')

print(len(data))

data['year'] = data['date'].apply(lambda x: int(str(x)[:4]))
data['date'] = pd.to_datetime(data['date'])

print('start:', data['date'].min())
print('end:', data['date'].max(), '\n')

print('nulls:')
print(data.isnull().sum())

21613
start: 2014-05-02 00:00:00
end: 2015-05-27 00:00:00 

nulls:
id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
year             0
dtype: int64


In [90]:
# p = figure(x_range=(-13630000, -13610000), y_range=(6038000, 6046000),
#            x_axis_type="mercator", y_axis_type="mercator")
# p.add_tile(CARTODBPOSITRON_RETINA)

# colors_dict = {2014: '#008000', 2015: '#FF0000'}

# for year in data['year'].unique():
    
#     coords = data.loc[data['year'] == year, ['price', 'lat', 'long']]
#     coords['merc'] = coords.apply(lambda x: coords_to_plot(x['lat'], x['long']), axis=1).values
    
#     for coord in coords.index[:500]:
        
#         x, y, price = coords.loc[coord]['merc'][0], coords.loc[coord]['merc'][1], coords.loc[coord]['price']

#         p.circle(x = x,
#                  y = y,
#                  size=price/100000,
#                  fill_alpha=.2,
#                  line_color = colors_dict[year],
#                  fill_color = colors_dict[year]
#                 )

# output_notebook()
# show(p)

In [91]:
def normalize(coords_price):
    cord_min = coords_price.min()
    cord_max = coords_price.max()
    def scale(cord):
        return (cord - cord_min)/(cord_max-cord_min)
    return coords_price.apply(lambda x: scale(x))

In [92]:
def make_json(row):
    return {'lat': row['lat'], 'long': row['long'], 'value': round(row['normed'], 6)} 

In [93]:
# Make sample data
coords = data.loc[data['year'] == 2014, ['price', 'lat', 'long']]
coords['normed'] = normalize(coords['price'])
coords = coords[:2000]

data = list(coords.apply(lambda x: make_json(x), axis=1).values)

with open('data.json', 'w') as fp:
    json.dump(data, fp)

In [112]:
cluster_lat_lon = get_cluster_dict(pd.DataFrame(data), 'lat', 'long', .5, 5)
# cluster_lat_lon
plot_weights(data)

KeyError: "['lat' 'long'] not in index"

# Not used

In [ ]:
# # No lat/long
# data = pd.read_csv('/Users/sam/Downloads/seattle-use-of-force/use-of-force.csv')
# data

In [ ]:
# # No lat/long
# data = pd.read_csv('/Users/sam/Downloads/seattle-rent-and-income-restricted-housing/rent-and-income-restricted-housing.csv')
# data

# Land use permits

In [113]:
data = pd.read_csv('./seattle-land-use-permits/land-use-permits.csv')
len(data)

for col in [col for col in data.columns if 'Date' in col]:
    try:
        data[col] = pd.to_datetime(data[col])
    except:
        pass
    
    print(col, 'min:', data[col].min())
    print(col, 'max:', data[col].max())
    
data.dtypes

Application Date min: 1992-01-30 00:00:00
Application Date max: 2018-04-27 00:00:00
Decision Date min: 1992-12-24 00:00:00
Decision Date max: 2018-04-27 00:00:00
Issue Date min: 2002-08-06 00:00:00
Issue Date max: 2018-04-27 00:00:00


Application/Permit Number                   int64
Permit Type                                object
Address                                    object
Description                                object
Category                                   object
Decision Type                              object
Design Review Included                     object
Value                                     float64
Applicant Name                             object
Application Date                   datetime64[ns]
Decision Date                      datetime64[ns]
Appealed?                                  object
Issue Date                         datetime64[ns]
Status                                     object
Contractor                                 object
Permit and Complaint Status URL            object
Latitude                                  float64
Longitude                                 float64
Location                                   object
dtype: object

In [114]:
data.isnull().sum()

Application/Permit Number              0
Permit Type                         2364
Address                               22
Description                           20
Category                            1519
Decision Type                       1359
Design Review Included              2840
Value                                  0
Applicant Name                        95
Application Date                    3848
Decision Date                       5303
Appealed?                              0
Issue Date                          5933
Status                              1258
Contractor                         12581
Permit and Complaint Status URL        0
Latitude                              27
Longitude                             27
Location                              13
dtype: int64

In [ ]:
# Applied
# Decisions/Applied
# Issues/Decisions
# Is there Issues/Applied?

# Find denied permits

In [ ]:
# Or find permits approved

In [ ]:
# for col in ['Permit Type', 'Category', 'Value', 'Applicant Name', 'Contractor']:
#     print(col)
#     print(data[col].unique())

In [ ]:
data['Category'].unique()

In [120]:
data.loc[data['Category'] == 'COMMERCIAL'].isnull().sum()

Application/Permit Number             0
Permit Type                         534
Address                               0
Description                           0
Category                              0
Decision Type                       307
Design Review Included              623
Value                                 0
Applicant Name                       11
Application Date                   1005
Decision Date                      1364
Appealed?                             0
Issue Date                         1443
Status                              296
Contractor                         2373
Permit and Complaint Status URL       0
Latitude                              1
Longitude                             1
Location                              0
dtype: int64

In [124]:
cluster_lat_lon = get_cluster_dict(data.loc[data['Category'] == 'COMMERCIAL'].dropna(), 'Latitude', 'Longitude', .5, 2)
# cluster_lat_lon
plot_weights(data)

Number of clusters: 54


Loading BokehJS ...

In [128]:
# data.loc[data['Category'] == 'MULTIFAMILY']
cluster_lat_lon = get_cluster_dict(data.loc[data['Category'] == 'MULTIFAMILY'].dropna(), 'Latitude', 'Longitude', .5, 2)
plot_weights(data)

Number of clusters: 44


Loading BokehJS ...